# Getting those coding RNAs's, lncRNAs's and their partnerRNA transcripts 's sequences with several coding potential tools

# Script content:

* [<font size="4">Pipeline overview</font>](#Pipeline-description):
    <br>

    - [<font size="3">Step 1: Preprocessing of input data</font>](#STEP-1:Preprocessing-of-input-data) 
     <br>

    - [<font size="3">Step 2: *de novo* assembly of transcripts</font>](#STEP-2:de-novo-assembly-of-transcripts)
    <br>

    - [<font size="3">Step 3: Transcriptome quality assessment</font>](#STEP-3:Transcriptome-quality-assessment)
    <br>

    - [<font size="3">Step 4: Merging assemblies</font>](#STEP-4:Merging-assemblies)
    <br>

    - [<font size="3">Step 5: Coding potential</font>](#STEP-5:Coding-potential)
    <br>
    
    - [<font size="3">Step 6: Annotation</font>](#STEP-6:Annotation)
    <br>

*  [<font size="4">Contact</font>](#Contact)
   <br>

*  [<font size="4">References</font>](#References)
   <br>

*  [<font size="4">Keynotes</font>](#Keynote)


# Required libraries

In [2]:
from csv import writer as write
import os
import pandas as pd
from Bio import SeqIO
import numpy as np
base_path= os.getcwd()

At first, those  analyzed transcripts with PlEK, CPC2, PORTRAIT, that search for coding potential in the overall transcriptome, need to be converted into dataframes, in order to be manipulated further:

# Non-coding and coding transcripts from PLEK (Li et al. 2014):

Convert PLEK's file into a dataframe:

In [4]:
PLEK=pd.read_csv('PLEK/nc_PLEK',sep='\t') 
PP=[PLEK.columns[0],PLEK.columns[2]]
PLEK.columns=['label','s','ID']
PPLEK=PLEK.append({'ID' : PP[1] , 'label' : PP[0]} , ignore_index=True)
PLEK2=PPLEK[['ID','label']]
print(PLEK2.shape)
PLEK2.tail()

(424, 2)


,ID,label
419,>rna_337,Non-coding
420,>rna_338,Non-coding
421,>rna_346,Non-coding
422,>rna_347,Non-coding
423,>idb_0,Non-coding


# Non-coding and coding transcripts from CPC2 (Kang et al. 2017):

Convert CPC2's file into a dataframe:

In [5]:
CPC2=pd.read_csv('CPC2/Purp_global', sep="\t",usecols=('#ID','label')) 
CPC2.columns=['ID','label']
print(CPC2.shape)
CPC2.tail()

(424, 2)


,ID,label
419,rna_336,noncoding
420,rna_337,noncoding
421,rna_338,noncoding
422,rna_346,noncoding
423,rna_347,noncoding


# Non-coding and coding transcripts from PORTRAIT (Arrial et al. 2009):

Convert PORTRAIT's file into a dataframe:

In [6]:
PORTRAIT= pd.read_csv('PORTRAIT/Purp_global.fasta_results_all.scores',sep=':') 
PR=[PORTRAIT.columns[0],PORTRAIT.columns[1]]
PORTRAIT.columns=['ID','label','score']
PPORTRAIT=PORTRAIT.append({'ID' : PR[0] , 'label' : PR[1]} , ignore_index=True)
PORTRAIT2=PPORTRAIT[['ID','label']]
print(PORTRAIT2.shape)
PORTRAIT2.tail()

(424, 2)


,ID,label
419,>rna_331,0
420,>rna_335,1
421,>rna_336,0
422,>rna_347,0
423,>idb_2,0


In the next object called 'Trans', the raw transcriptome, in which it's coding potencial was analyzed, should be stored:

In [7]:
Trans=list(SeqIO.parse(os.path.join(base_path,'raw_assemblies/Purp_global.fasta'),'fasta'))

# Main functions:

**i.** Function to distribute coding and non-coding transcripts into separated lists. It receives each program's dataframe, previously generated, and the 'busco' parameter, either 'coding' or 'noncoding', which clarifies what you're looking for, coding or non-coding transcript list, respectively. This function returns a list with the coding or non-coding transcript's identifiers.

In [8]:
def decoding(dataframe,busco):
    noncoding_list=[]
    coding_list=[]
    i=0
    f=len(dataframe)
    x=(len(dataframe)-1)
    while i<f:
        a=dataframe.iloc[i]['label']           
        if a=='coding' or a==1:
            coding_list.append(dataframe.iloc[i]['ID'])
            i+=1
        else:
            noncoding_list.append(dataframe.iloc[i]['ID'])
            i+=1
    if i==f and busco=='coding':
        return coding_list
    elif i==f and busco=='noncoding':
        return noncoding_list

**ii.**Function to change ID's format from PLEK and PORTRAIT, with means to be comparable with those from CPC2. The only receiving parameter is the spawned list with the function decoding(). It returns a modified list, equivalent with CPC2's IDs.

In [9]:
def formatting(lista):
    sepList=[]
    i=0
    n=0
    while i<len(lista):
        sepList.append(lista[i].split()[0].strip('>'))
        i=i+1
    return sepList

**Coding transcripts lists from each program:**

In [10]:
cCPC2=decoding(CPC2,'coding')
cPLEK=decoding(PLEK2,'coding')
cPORTRAIT=formatting(decoding(PORTRAIT2,'coding'))

**Non-coding transcripts lists from each program:**

In [11]:
ncCPC2=decoding(CPC2,'noncoding')
ncPLEK=formatting(decoding(PLEK2,'noncoding'))
ncPORTRAIT=formatting(decoding(PORTRAIT2,'noncoding'))

For instance, you should find which list has the minimal length, or no length at all, in order to use it as the threshold for the function concatenate(), likeso:

In [12]:
def length(list1,list2,list3):
    le1=len(list1)
    le2=len(list2)
    le3=len(list3)
    if le1==0:
        if le2<le3:
            return 'Caution, list1 equals 0 and list2 has the lowest length'
        else:
            return 'Caution, list1 equals 0 and list3 has the lowest length'
    elif le2==0:
        if le1<le3:
            return 'Caution, list2 equals 0 and list1 has the lowest length'
        else:
            return 'Caution, list2 equals 0 and list3 has the lowest length'
    elif le3==0:
        if le2<le1:
            return 'Caution, list3 equals 0 and list2 has the lowest length'
        else:
            return 'Caution, list3 equals 0 and list1 has the lowest length'
    elif le1<le3 and le1<le2:   
        return 'The list1 has the lowest length'
    elif le2<le3 and le2<le1:
        return 'The list2 has the lowest length'
    else:
        return 'The list3 has the lowest length'    

**iii.** Function to concatenate common coding and non-coding lists, from the three outputs of the aforementioned
programs,into two lists, duplicates presence not allowed, concatri(). It receives the three lists, but you should pay attention to their order. The only requirement whatsoever is the first list, that must be the one with the lowest length. In addition, if you noticed with the function length() a list with length 0, you should use instead, the function concadu() for the two lists with non zero length:

In [13]:
def concatri(list1,list2,list3):
    l1=len(list1)
    l2=len(list2)
    l3=len(list3)
    final_list=[]
    for i in range(l1):
        for x in range(l2):
            if list1[i]==list2[x]:
                for j in range(l3):
                    if list1[i]==list3[j]:
                        final_list.append(list1[i])
    return final_list 

In [14]:
def concadu(list1,list2):
    l1=len(list1)
    l2=len(list2)
    final_list=[]
    for i in range(l1):
        for x in range(l2):
            if list1[i]==list2[x]:
                final_list.append(list1[i])
    return final_list 

**Identifying common coding transcripts from PLEK,PORTRAIT and CPC2**

In [15]:
length(cPLEK,cPORTRAIT,cCPC2)

'Caution, list1 equals 0 and list3 has the lowest length'

So, as the list 1 and list 3 lengths equal 0  and the lowest number, respectively, you should use the function concadu(), with the list1 parameter being the list with the lowest length, in this case, cCPC2:

In [16]:
cc=concadu(cCPC2,cPORTRAIT) #cc means common coding transcripts

**Identifying common non-coding transcripts from PLEK,PORTRAIT and CPC2**

In [17]:
length(ncPLEK,ncPORTRAIT,ncCPC2)

'The list2 has the lowest length'

As the list2 has the lowest length overall, you should use it with the function concatri(), as the first parameter likeso:

In [18]:
nc=concatri(ncPORTRAIT,ncPLEK,ncCPC2) #nc means common non-coding transcripts

**iv**.Function to identify those non-coding and coding transcripts in the raw transcriptome. It receives the raw transcriptome and the list with the coding or non-coding sequences IDs. Its output is a list with the coding and non-coding IDs's and respective sequences's.

In [19]:
def onT(record,lista):
    transcriptome_nc=[]
    f=len(lista)
    i=0
    k=0
    while i<f:
        if  lista[i]==record[k].id:
            a='>'+record[k].id+'\n'+record[k].seq
            a.strip('Seq(')
            transcriptome_nc.append(a)
            i=i+1
            k=0
        else:
            k+=1
    return transcriptome_nc                      

**Output de la función onT:**

In [20]:
cFin=onT(Trans,cc)
ncFin=onT(Trans,nc)

**v**. Function to convert both previously lists, cFin and ncFin, into fasta files. It receives each of these lists and a parameter, which could be: 'coding', 'noncoding' or 'other', meaning which kind of transcriptome is desired:


In [26]:
def toFa(lista,param):
    if param=='coding':
        f=open('Coding_global_Transcriptome.fasta','w+')
    elif param=='noncoding':
        f=open('Noncoding_global_Transcriptome.fasta','w+')
    elif param=='other':
        f=open('Other.fasta','w+')
    for i in range(len(lista)):
        f.write((str(lista[i])).strip('.'))
        f.write('\n')
    f.close()
    return 'Transcriptome assembled!'

**To get the coding global transcriptome:**

In [27]:
toFa(cFin,'coding')

'Transcriptome assembled!'

**To get the non-coding global transcriptome:**

In [28]:
toFa(ncFin,'noncoding')

'Transcriptome assembled!'

# LncRNAs analysis from FEELnc (Wucher et al. 2017):

FEELnc (Wucher et al. 2017), is a software to identify and anotate long non-coding RNAs (lncRNAs) into diferent classes, and regarding their closeness with  neighboring coding genes in a reference genome or with other aproaches.

At first, you should load your FEELnc ouput into a dataframe:

In [3]:
FEEL= pd.read_csv('lncClases.txt',sep='\t') 
print(FEEL.shape)
FEEL.head()

(109, 10)


,isBest,lncRNA_gene,lncRNA_transcript,partnerRNA_gene,partnerRNA_transcript,direction,type,distance,subtype,location
0,1,g2783,g2783.t1,g2782,g2782.t1,antisense,intergenic,2764,convergent,downstream
1,1,g11234,g11234.t1,g11235,g11235.t1,sense,intergenic,563,same_strand,downstream
2,1,g10132,g10132.t1,g10130,g10130.t1,sense,intergenic,4628,same_strand,upstream
3,1,g9208,g9208.t1,g9207,g9207.t1,antisense,intergenic,51,convergent,downstream
4,1,g8889,g8889.t1,g8890,g8890.t1,sense,intergenic,7114,same_strand,upstream


**vi.**Function to filtrate only good quality lncRNAs. It receives the dataframe and its output are those lncRNAs with the best metrics based upon the variable 'isBest' (Wucher et al. 2017):

In [4]:
def Filtering(df):
    data=[]
    for i in range(len(df)):
        if df['isBest'][i]== 1:
            data.append([df['isBest'][i],df['lncRNA_gene'][i],df['partnerRNA_gene'][i],
                        df['partnerRNA_transcript'][i],df['direction'][i],df['type'][i],
                        df['distance'][i],df['subtype'][i],df['location'][i]]) 
    return data

**Function Filtering Output:**

In [7]:
FIFEEL= pd.DataFrame(Filtering(FEEL), columns = ['isBest','lncRNA_gene','partnerRNA_gene',
                                                 'partnerRNA_transcript','direction','type','distance',
                                               'subtype','location']) 
print(FIFEEL.shape)
FIFEEL.head()

(103, 9)


,isBest,lncRNA_gene,partnerRNA_gene,partnerRNA_transcript,direction,type,distance,subtype,location
0,1,g2783,g2782,g2782.t1,antisense,intergenic,2764,convergent,downstream
1,1,g11234,g11235,g11235.t1,sense,intergenic,563,same_strand,downstream
2,1,g10132,g10130,g10130.t1,sense,intergenic,4628,same_strand,upstream
3,1,g9208,g9207,g9207.t1,antisense,intergenic,51,convergent,downstream
4,1,g8889,g8890,g8890.t1,sense,intergenic,7114,same_strand,upstream


**vii.** Function to create a new file with specific characteristics from the output of FEELnc. It receives the filtered dataframe from the function filtering(), and the specific column's data to get.

In [9]:
def toTxt(df,column):
    f=open('Output_FEEL_lncRNAs.txt','w+')
    for i in range(len(df)):
        f.write(df[column][i])
        f.write('\n')
    f.close()
    return 'Dataframe to STRING'

**Example with the function toTxt(), to get the IDs of the column 'partnerRNA_transcript' from the filtered dataframe FIFEEL:**

In [10]:
toTxt(FIFEEL,'partnerRNA_transcript')

'Dataframe to STRING'

# Getting lncRNAs IDs's  and their partnerRNA transcripts's (PRNAs) sequences from the genome annotations:

First, load the fasta file from the genome annotation into a list : 

In [21]:
Anno=list(SeqIO.parse(os.path.join(base_path,'SeqsFromGFF/protein.fa'),'fasta'))

Then, create a new dataframe but only using the good quality FEELnc IDs with a specific column:

In [24]:
FEELpd=pd.read_csv('Output_FEEL_lncRNAs.txt')
FF=FEELpd.columns[0]
FEELpd.columns=['partnerRNA_transcript']
PT=FEELpd.append({'partnerRNA_transcript' : FF} , ignore_index=True)
print(PT.shape)
PT.head()

(103, 1)


,partnerRNA_transcript
0,g11235.t1
1,g10130.t1
2,g9207.t1
3,g8890.t1
4,g7644.t1


**viii.**Function to  convert a specific column from the FEELnc dataframe previously chosen, into  a list in order to find their sequences in the provided reference genome.

In [29]:
def dftoList(df,column):
    Lista=[]
    for i in range(len(df)):
        Lista.append(df[column][i])
    return (Lista)

**Output function dftoList**:

In [44]:
LiPT=dftoList(PT,'partnerRNA_transcript')

**ix.** Finally, a new object, ncTrans, will be generated with the next function,onG(). It looks for the specific IDs,
from the list brought by the function dftoList, in the genome annotation file. Its output is a new list having the desired ID's with their respective sequences alongside.

In [46]:
def onG(record,lista):
    transcriptome_nc=[]
    f=len(lista)
    i=0
    k=0
    while i<f:
        if  lista[i]==(record[k].id).strip('.t1') or lista[i]==(record[k].id):
            a='>'+record[k].id+'\n'+record[k].seq
            a.strip('Seq(')
            transcriptome_nc.append(a)
            k=0
            i+=1
        else:
            if k==len(Anno)-1:
                i+=1
                k=0
            else:
                k+=1       
    return transcriptome_nc          

In [47]:
ncTrans=onG(Anno,LiPT)

Finally, you should put these IDs and sequences in a fasta file format.At the end, you should have a fasta file with either the lncRNAs sequences or the PRNAs sequences to be further analyzed. 

In [62]:
toFa(ncTrans,'other')

'Transcriptome assembled!'

These script was developed by mi cuenta bla bla, with aid from the different available coding potential tools mentioned below.

# References

1.Arrial, R. T., Togawa, R. C., & de M Brigido, M. (2009). Screening non-coding RNAs in transcriptomes from neglected species using PORTRAIT: case study of the pathogenic fungus Paracoccidioides brasiliensis. BMC bioinformatics, 10(1), 239.

2.Li, A., Zhang, J., & Zhou, Z. (2014). PLEK: a tool for predicting long non-coding RNAs and messenger RNAs based on an improved k-mer scheme. BMC bioinformatics, 15(1), 311.

3.Kang, Y. J., Yang, D. C., Kong, L., Hou, M., Meng, Y. Q., Wei, L., & Gao, G. (2017). CPC2: a fast and accurate coding potential calculator based on sequence intrinsic features. Nucleic acids research, 45(W1), W12-W16.

4.Wucher, V., Legeai, F., Hedan, B., Rizk, G., Lagoutte, L., Leeb, T., ... & Cirera, S. (2017). FEELnc: a tool for long non-coding RNA annotation and its application to the dog transcriptome. Nucleic acids research, 45(8), e57-e57.

5.Kumar, H., Srikanth, K., Park, W., Lee, S. H., Choi, B. H., Kim, H., ... & Jung, J. Y. (2019). Transcriptome analysis to identify long non coding RNA (lncRNA) and characterize their functional role in back fat tissue of pig. Gene, 703, 71-82.
